In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [4]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import InMemorySaver

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState  
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [6]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='ad8a3dcb-b1f8-48d9-bb1d-661ef7b9f835'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}, '__gemini_function_call_thought_signatures__': {'574d919d-96ad-4169-ad04-853b5c78ab8d': 'CvQBAXLI2nwATfAnL4LZv+5o0eB6iUgFI8WXBEmdam+GaE2wHPS93QNWd1dRNZwsqOxoCFb0XhbFCN9yQawzDNDMJKvNwjlrJzavPenDMlyOVcnd9H/BQI8+swyjN+CtJJsUSfdCWBsSSjQYdZTKXcHva8JSOp2FR0Ntu1GhdCXw4Eaaydl1/IoKU+V9CU502Bfs1GJFlU6MfVIc6fuudAbwsAb9vxRY2DLD2ZXYWqTnhB+0Po2t7c5GaEw+1ihIgoE76yoOy9TX6kwbAXzek7BimJWnJE06CNfMKI2FEV1ZRyxpS4A3yNc/zr+ie5XY+p1/f4ab/w=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bfac4-7530-78c0-8f7d-727fbc28214b-0', tool_calls=[{'name': 'update_favourite_colour'

In [7]:
# Pass update during invoke

response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='f1f05a8d-249b-4303-801e-430f780aa80c'),
              AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bfac4-81b6-7601-bc37-bf2a5735248b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 65, 'output_tokens': 16, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}})]}


## Read state

In [8]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [9]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is black")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'black',
 'messages': [HumanMessage(content='My favourite colour is black', additional_kwargs={}, response_metadata={}, id='7ed2d1db-0d72-4e5c-8ce8-9accec1423e5'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "black"}'}, '__gemini_function_call_thought_signatures__': {'2b81c29d-9943-4f98-87e1-63e68dfb5653': 'CvQBAXLI2nyZdB837St7JxIi/aicOm8vl7bNDcCqbJJADBSVIiZS3l6AIcgVk8ahpYc+83Q2ci+YjwuDzdyONFQyAWRv4FbNQLDMEd153HL5C3lMYsO25KrtB82It30crPHY3QOxX1czDWRgYTAwqaxCtCcFU17GX6izkD1x8KFgs38WP0udPv0cwv7ty1G01LRifqtayjoybbxmQukih2T1YZsnEd9dG69haqO/qRJVm2CY2o5jVZoHriGce9rwcqONRbYqPOmsSRLP/VDeuM/6cQhMkq257zGFwqnBayaecZxMzM8QkBy9Q9/lu2jeWreZYtTpQQ=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bfac4-85e6-7782-8d19-17ccf09aca9e-0', tool_calls=[{'name': 'update_favourite_colour'

In [10]:
# Updated thread

response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'black',
 'messages': [HumanMessage(content='My favourite colour is black', additional_kwargs={}, response_metadata={}, id='7ed2d1db-0d72-4e5c-8ce8-9accec1423e5'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "black"}'}, '__gemini_function_call_thought_signatures__': {'2b81c29d-9943-4f98-87e1-63e68dfb5653': 'CvQBAXLI2nyZdB837St7JxIi/aicOm8vl7bNDcCqbJJADBSVIiZS3l6AIcgVk8ahpYc+83Q2ci+YjwuDzdyONFQyAWRv4FbNQLDMEd153HL5C3lMYsO25KrtB82It30crPHY3QOxX1czDWRgYTAwqaxCtCcFU17GX6izkD1x8KFgs38WP0udPv0cwv7ty1G01LRifqtayjoybbxmQukih2T1YZsnEd9dG69haqO/qRJVm2CY2o5jVZoHriGce9rwcqONRbYqPOmsSRLP/VDeuM/6cQhMkq257zGFwqnBayaecZxMzM8QkBy9Q9/lu2jeWreZYtTpQQ=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bfac4-85e6-7782-8d19-17ccf09aca9e-0', tool_calls=[{'name': 'update_favourite_colour'